# Amazon Sales Report — End‑to‑End Analysis
Run the cells below to reproduce the analysis, KPIs, and charts. (Matplotlib only; no seaborn.)

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil import parser as dtparser

csv_path = Path('/mnt/data/Amazon Sale Report (1).csv')  # adjust if needed
out_dir = Path('/mnt/data/amazon_sales_output')
out_dir.mkdir(parents=True, exist_ok=True)

def _safe_lower_strip(s):
    if pd.isna(s): return s
    return str(s).strip()

def parse_date_safe(x):
    if pd.isna(x): return pd.NaT
    try:
        return pd.to_datetime(dtparser.parse(str(x), dayfirst=False))
    except Exception:
        try:
            return pd.to_datetime(x, errors='coerce')
        except Exception:
            return pd.NaT

def to_number(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int,float,np.number)): return float(x)
    s = str(x).replace(',', '').replace('₹','').replace('$','').replace('€','').strip()
    try: return float(s)
    except Exception: return np.nan

df = pd.read_csv(csv_path)
df.columns = [c.strip() for c in df.columns]
df = df.drop_duplicates()

date_col = 'Date' if 'Date' in df.columns else None
if date_col:
    df['parsed_date'] = df[date_col].apply(parse_date_safe)
    df['order_date'] = df['parsed_date'].dt.date
    df['year'] = df['parsed_date'].dt.year
    df['month'] = df['parsed_date'].dt.to_period('M').astype(str)
    df['week'] = df['parsed_date'].dt.to_period('W').astype(str)
else:
    df['parsed_date'] = pd.NaT

df['Qty_num'] = pd.to_numeric(df.get('Qty', np.nan), errors='coerce')
df['Amount_num'] = pd.Series(df.get('Amount', np.nan)).apply(to_number)

for col in ['Status','Fulfilment','Sales Channel','Category','Size','Courier Status',
            'ship-city','ship-state','ship-country','currency','B2B','fulfilled-by']:
    if col in df.columns:
        df[col] = df[col].apply(_safe_lower_strip)

def status_bucket(s):
    if pd.isna(s): return 'unknown'
    s = s.lower()
    if any(k in s for k in ['cancel','refunded','return']): return 'non_revenue'
    if any(k in s for k in ['pending','on-hold']): return 'pending'
    return 'completed'

df['status_bucket'] = df.get('Status', 'completed')
if 'Status' in df.columns:
    df['status_bucket'] = df['Status'].apply(status_bucket)
else:
    df['status_bucket'] = 'completed'

df['gross_revenue'] = df['Amount_num'].fillna(0.0)
df['units'] = df['Qty_num'].fillna(0.0)
df['net_revenue'] = np.where(df['status_bucket'].eq('completed'), df['gross_revenue'], 0.0)
df['order_id'] = df.get('Order ID', pd.Series(range(len(df)))).astype(str)

df.to_csv(out_dir/'cleaned_sales.csv', index=False)
df.head(5)


## KPIs

In [ ]:
completed = df[df['status_bucket'].eq('completed')]
kpis = {
    'total_orders': completed['order_id'].nunique(),
    'total_units': float(completed['units'].sum()),
    'gross_revenue': float(df['gross_revenue'].sum()),
    'net_revenue': float(completed['net_revenue'].sum()),
    'avg_order_value': float(completed.groupby('order_id')['net_revenue'].sum().mean() if len(completed) else 0.0),
    'avg_selling_price': float((completed['net_revenue'].sum() / completed['units'].sum()) if completed['units'].sum() > 0 else 0.0),
    'return_cancel_rate_%': float(100.0 * (df['status_bucket'].eq('non_revenue').sum() / len(df)) if len(df) else 0.0)
}
kpis_df = pd.DataFrame([kpis])
kpis_df.to_csv(out_dir/'kpis.csv', index=False)
kpis_df


## Time Series

In [ ]:
if df['parsed_date'].isna().all():
    print("No valid dates available for time series.")
else:
    daily = completed.groupby('order_date').agg(net_revenue=('net_revenue','sum'),
                                                units=('units','sum'),
                                                orders=('order_id','nunique')).reset_index()
    monthly = completed.groupby('month').agg(net_revenue=('net_revenue','sum'),
                                             units=('units','sum'),
                                             orders=('order_id','nunique')).reset_index()
    daily.to_csv(out_dir/'ts_daily.csv', index=False)
    monthly.to_csv(out_dir/'ts_monthly.csv', index=False)
    
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(pd.to_datetime(daily['order_date']), daily['net_revenue'])
    plt.title('Daily Net Revenue'); plt.xlabel('Date'); plt.ylabel('Net Revenue')
    plt.tight_layout(); plt.savefig(out_dir/'ts_daily_net_revenue.png'); plt.show()
    
    plt.figure()
    plt.bar(pd.to_datetime(monthly['month'].astype(str)), monthly['net_revenue'])
    plt.title('Monthly Net Revenue'); plt.xlabel('Month'); plt.ylabel('Net Revenue')
    plt.tight_layout(); plt.savefig(out_dir/'ts_monthly_net_revenue.png'); plt.show()


## Categorical Breakdowns

In [ ]:
breakdowns = {}
for col in ['Category','Size','Fulfilment','Sales Channel','ship-city','ship-state','ship-country','Courier Status','currency','fulfilled-by','B2B']:
    if col in df.columns:
        tmp = completed.groupby(col).agg(net_revenue=('net_revenue','sum'),
                                         units=('units','sum'),
                                         orders=('order_id','nunique')).reset_index().sort_values('net_revenue', ascending=False)
        breakdowns[col] = tmp
        tmp.to_csv(out_dir / f"by_{col.replace(' ', '_').replace('-', '_')}.csv", index=False)
        head = tmp.head(10)
        plt.figure()
        plt.bar(head[col].astype(str), head['net_revenue'])
        plt.title(f"Top {min(10, len(head))} by {col} — Net Revenue")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout(); plt.savefig(out_dir / f"by_{col.replace(' ', '_').replace('-', '_')}_top10.png"); plt.show()
        
list(breakdowns.keys())


## RFM-like Segmentation (city/state proxy)

In [ ]:
if {'ship-city','ship-state','parsed_date'}.issubset(df.columns) and not df['parsed_date'].isna().all():
    now = df.loc[df['status_bucket'].eq('completed'),'parsed_date'].max() + pd.Timedelta(days=1)
    completed2 = df[df['status_bucket'].eq('completed') & df['parsed_date'].notna()].copy()
    key = completed2[['ship-city','ship-state']].fillna('unknown').agg('|'.join, axis=1)
    grp = completed2.assign(segment=key).groupby('segment')
    r = grp['parsed_date'].max().apply(lambda d: (now - d).days)
    f = grp['order_id'].nunique()
    m = grp['net_revenue'].sum()
    rfm = pd.DataFrame({'R': r, 'F': f, 'M': m}).reset_index()
    
    rfm['R_score'] = pd.qcut(-rfm['R'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
    rfm['F_score'] = pd.qcut(rfm['F'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
    rfm['M_score'] = pd.qcut(rfm['M'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
    rfm['RFM_total'] = rfm[['R_score','F_score','M_score']].sum(axis=1)
    def bucket(total):
        if total >= 13: return 'Champions'
        if total >= 10: return 'Loyal/High-Value'
        if total >= 7: return 'Potential Loyalist'
        if total >= 5: return 'Needs Attention'
        return 'At Risk'
    rfm['Segment'] = rfm['RFM_total'].apply(bucket)
    rfm.to_csv(out_dir/'rfm_segments.csv', index=False)
    
    counts = rfm['Segment'].value_counts().reset_index()
    plt.figure()
    plt.bar(counts['index'].astype(str), counts['Segment'])
    plt.title('RFM Segment Distribution'); plt.xticks(rotation=30, ha='right')
    plt.tight_layout(); plt.savefig(out_dir/'rfm_segment_distribution.png'); plt.show()
else:
    print("Not enough fields for RFM proxy segmentation.")


## Save quick insights

In [ ]:
insights = []
insights.append(f"Total Net Revenue: {kpis['net_revenue']:.2f}")
insights.append(f"Total Orders: {kpis['total_orders']} | Units: {kpis['total_units']:.0f}")
insights.append(f"AOV: {kpis['avg_order_value']:.2f} | ASP: {kpis['avg_selling_price']:.2f}")
insights.append(f"Return/Cancel Rate: {kpis['return_cancel_rate_%']:.2f}%")

if 'month' in df.columns and not df['parsed_date'].isna().all():
    monthly = completed.groupby('month')['net_revenue'].sum().reset_index().sort_values('net_revenue', ascending=False)
    if not monthly.empty:
        insights.append(f"Best Month by Net Revenue: {monthly.iloc[0]['month']}")

if 'Category' in df.columns and 'net_revenue' in df.columns:
    cat = breakdowns.get('Category')
    if cat is not None and not cat.empty:
        insights.append(f"Top Category: {cat.iloc[0]['Category']} — {cat.iloc[0]['net_revenue']:.2f} net revenue")

if 'ship-state' in df.columns:
    state = breakdowns.get('ship-state')
    if state is not None and not state.empty:
        insights.append(f"Top State: {state.iloc[0]['ship-state']} — {state.iloc[0]['net_revenue']:.2f} net revenue")

(Path('/mnt/data/amazon_sales_output') / 'insights.txt').write_text("\n".join(insights), encoding='utf-8')
insights
